# Conditional Generative Adversarial Network



### Loading datset and preprocessing

In [1]:
from tensorflow.keras.datasets import mnist

# load mnist x_train
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [2]:
import numpy as np
from collections import Counter
from tensorflow.keras.utils import to_categorical

""" Preprocessing """

# create batch_dim
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)

# convert to keras compatible float
x_train = x_train.astype(np.float32)

# count different classes
num_classes = len(Counter(y_train).keys())

# create to_categorical labels
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# normalization to inputs between [-1, 1]
MAX = np.max(x_train)
x_train = (x_train / (MAX/2)) - 1

print(f"Data_shape: {x_train.shape} - Input_range: [{np.min(x_train)}, {np.max(x_train)}]")

Data_shape: (60000, 28, 28, 1) - Input_range: [-1.0, 1.0]


### Discriminator Network


In [3]:
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

img_shape = x_train.shape[1:]

img = Input(shape=(img_shape))
label = Input(shape=(num_classes,))
img_flatten = Flatten()(img)

x = Concatenate()([img_flatten, label])
x = Dense(units=512)(x)
x = LeakyReLU(alpha=0.2)(x)
x = Dropout(rate=0.25)(x)
x = Dense(units=512)(x)
x = LeakyReLU(alpha=0.2)(x)
x = Dropout(rate=0.25)(x)
x = Dense(units=1)(x)
d_pred = Activation("sigmoid")(x)

DISCRIMINATOR = Model(inputs=[img, label], outputs=d_pred)
DISCRIMINATOR.summary()

DISCRIMINATOR.compile(
    loss="binary_crossentropy",
    optimizer=Adam(learning_rate=0.0002, beta_1=0.5),
    metrics=["accuracy"]
)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
flatten (Flatten)               (None, 784)          0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 794)          0           flatten[0][0]                    
                                                                 input_2[0][0]                

### Generator Net

In [4]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Reshape

z_dim = 100

noise = Input(shape=(z_dim,))
label = Input(shape=(num_classes,))

x = Concatenate()([noise, label])
x = Dense(units=512)(x)
x = LeakyReLU(alpha=0.2)(x)
x = BatchNormalization()(x)
x = Dense(units=1024)(x)
x = LeakyReLU(alpha=0.2)(x)
x = BatchNormalization()(x)
x = Dense(np.prod(img_shape))(x)
x = Activation("tanh")(x)
img = Reshape(img_shape)(x)

GENERATOR = Model(inputs=[noise, label], outputs=img)
GENERATOR.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 110)          0           input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 512)          56832       concatenate_1[0][0]        

### GAN Net

In [5]:
# combining DISCRIMINATOR and GENERATOR to GAN
noise_in = Input(shape=(z_dim,)) # noise in
label = Input(shape=(num_classes,)) # label in
img = GENERATOR([noise_in, label]) # generated image
DISCRIMINATOR.trainable = False
d_pred = DISCRIMINATOR([img, label]) # prediction if image input is real or fake
CGAN = Model(inputs=[noise_in, label], outputs=d_pred)

CGAN.compile(
    loss="binary_crossentropy",
    optimizer=Adam(learning_rate=0.0002, beta_1=0.5)
)

### Training

In [6]:
import matplotlib.pyplot as plt
import os

PATH = "generated_images"
try:
    os.makedirs(PATH)
except FileExistsError:
    pass

# creating labels for dataset
batch_size = 128
y_real = np.ones(shape=(batch_size, 1)) # 1 = 100% real
y_fake = np.zeros(shape=(batch_size, 1)) # 0 = 0% real

epochs=10_000

for epoch in range(1, epochs+1):
    # get random images from real dataset
    rand_idxs = np.random.randint(low=0, high=x_train.shape[0], size=batch_size)
    real_imgs = x_train[rand_idxs]
    classes = y_train[rand_idxs] # get label to image
    # generate fake images for training
    noise = np.random.normal(loc=0.0, scale=1.0, size=(batch_size, z_dim)) # noise vector (32, 100)
    fake_imgs = GENERATOR([noise, classes], training=False)
    # train DISCRIMINATOR
    d_loss_real = DISCRIMINATOR.train_on_batch([real_imgs, classes], y_real)
    d_loss_fake = DISCRIMINATOR.train_on_batch([fake_imgs, classes], y_fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    # train GENERATOR
    noise = np.random.normal(loc=0.0, scale=1.0, size=(batch_size, z_dim))
    g_loss = CGAN.train_on_batch([noise, classes], y_real)

    # saving every 1000 steps
    if((epoch % 1000) == 0) or (epoch==1):
        print(
            f"{epoch} - D_loss: {round(d_loss[0], 4)}"
            f" D_acc: {round(d_loss[1], 4)}"
            f" G_loss: {round(g_loss, 4)}"
        )

        rows, cols = 2, 5
        noise = np.random.normal(loc=0.0, scale=1.0, size=(rows * cols, z_dim))
        labels = np.random.randint(0, num_classes, 10)
        labels_categorical = to_categorical(labels, num_classes=num_classes)
        gen_imgs = GENERATOR.predict([noise, labels_categorical]) # generating image
        gen_imgs = 0.5 * gen_imgs + 0.5
        fig, axs = plt.subplots(rows, cols)
        cnt = 0
        for i in range(rows):
            for j in range(cols):
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap="gray")
                axs[i, j].set_title(f"Digit: {labels[cnt]}")
                axs[i, j].axis("off")
                cnt +=1
        img_name = f"{epoch}.png"
        fig.savefig(os.path.join(PATH, img_name))
        plt.close()

1 - D_loss: 0.8023 D_acc: 0.4922 G_loss: 0.8261
1000 - D_loss: 0.5604 D_acc: 0.7266 G_loss: 0.9761
2000 - D_loss: 0.5225 D_acc: 0.7578 G_loss: 1.2688
3000 - D_loss: 0.6139 D_acc: 0.6719 G_loss: 1.082
4000 - D_loss: 0.6424 D_acc: 0.6211 G_loss: 0.9661
5000 - D_loss: 0.6575 D_acc: 0.5977 G_loss: 0.9516
6000 - D_loss: 0.6726 D_acc: 0.5664 G_loss: 0.916
7000 - D_loss: 0.6795 D_acc: 0.6133 G_loss: 0.8233
8000 - D_loss: 0.6898 D_acc: 0.5273 G_loss: 0.8383
9000 - D_loss: 0.6791 D_acc: 0.6094 G_loss: 0.8347
10000 - D_loss: 0.6918 D_acc: 0.5273 G_loss: 0.8173
